In [28]:
import numpy as np
import holoviews as hv
from holoviews import opts
import xarray as xr
import hvplot.xarray

In [47]:
import xarray as xr
import xrspatial as xrs
import datashader as ds
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
import geoviews as gv
import rioxarray as rx

In [48]:
#help(xrs.where())

In [56]:
data = rx.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif",masked =True, cache = False).squeeze().astype('float64')

RasterioIOError: HTTP response code: 404

In [50]:
data

<xarray.DataArray (y: 3612, x: 3612)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan]])
Coordinates:
    band         int64 1
  * y            (y) float64 19.0 19.0 19.0 19.0 19.0 ... 18.0 18.0 18.0 18.0
  * x            (x) float64 -65.0 -65.0 -65.0 -65.0 ... -64.0 -64.0 -64.0 -64.0
    spatial_ref  int64 0

In [51]:
data2 = xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif")

In [52]:
data2

<xarray.DataArray (band: 1, y: 3612, x: 3612)>
[13046544 values with dtype=float32]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 19.0 19.0 19.0 19.0 19.0 ... 18.0 18.0 18.0 18.0 18.0
  * x        (x) float64 -65.0 -65.0 -65.0 -65.0 ... -64.0 -64.0 -64.0 -64.0
Attributes:
    transform:              (0.0002777777780359644, 0.0, -65.0016666671822, 0...
    crs:                    +init=epsg:4269
    res:                    (0.0002777777780359644, 0.0002777777777870158)
    is_tiled:               1
    nodatavals:             (-3.4028230607370965e+38,)
    scales:                 (1.0,)
    offsets:                (0.0,)
    descriptions:           ('Layer_1',)
    AREA_OR_POINT:          Area
    BandDefinitionKeyword:  NONE
    DataType:               Elevation

In [53]:
data2.res

(0.0002777777780359644, 0.0002777777777870158)

In [54]:
data.attrs = data2.attrs

In [55]:
data

<xarray.DataArray (y: 3612, x: 3612)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan]])
Coordinates:
    band         int64 1
  * y            (y) float64 19.0 19.0 19.0 19.0 19.0 ... 18.0 18.0 18.0 18.0
  * x            (x) float64 -65.0 -65.0 -65.0 -65.0 ... -64.0 -64.0 -64.0 -64.0
    spatial_ref  int64 0
Attributes:
    transform:              (0.0002777777780359644, 0.0, -65.0016666671822, 0...
    crs:                    +init=epsg:4269
    res:                    (0.0002777777780359644, 0.0002777777777870158)
    is_tiled:               1
    nodatavals:             (-3.4028230607370965e+38,)
    scales:                 (1.0,)
    offsets:                (0.0,)
    descriptions:           ('Layer_1',)
    AREA_OR_POINT:          Area
    BandDefinitionKeyword:  NONE
    DataType:               Elevation

In [ ]:
data =data.where(data>-1e38)